# Additional features and functionalities of flower

## Overview:
In this tutorial, we present additional features and functionalities within flower, empowering users to customize their simulations according to their specific requirements. We will cover the following topics for `flower` modules:

**1) Molecule**

- Custom Molecule definition
- Polymer and CoPolymer Builders
  
**2) ForceField**

- Custom ForceFields (XML-based)
- Custom HOOMD Forces

**3) System**
   
- Custom assembly algorithms
- Mixture of molecules
- Reference values (auto scaling)

**4) Simulation**
- Simulation methods
- Resuming a simulation



## 1. Molecule

### Custom Molecule definition

You can define custom molecules in a couple of different ways:

1. Define molecules using their [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) string representation.
2. Utilize molecule files in formats such as `.mol` or `.sdf` to specify your custom molecules.
3. Create molecules from an [`mBuild`](https://mbuild.mosdef.org/en/stable/) compound or a [`GMSO`](https://gmso.mosdef.org/en/stable/) topology.
4. Customize molecules by creating a subclass of the `Molecule` class.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#### Option 1: Using the SMILES string of the molecule

In [ ]:
from flower.base import Molecule

benzoic_acid_mol = Molecule(num_mols=20, smiles="c1cc(C(O)=O)ccc1")

We can use the `mBuild` visualization function to visualize one of the 20 benzoic acid molecules.

In [ ]:
benzoic_acid_mol.molecules[0].visualize()

#### Option 2: Initialize molecule from a file

In [ ]:
phenol_mol = Molecule(num_mols=20, file="../flower/assets/molecule_files/IPH.mol2")

In [ ]:
phenol_mol.molecules[0].visualize()

#### Option 3: Start molecule from an [`mBuild`](https://mbuild.mosdef.org/en/stable/) compound or a [`GMSO`](https://gmso.mosdef.org/en/stable/) topology

In [ ]:
import mbuild as mb

mb_compound = mb.load("c1ccccc1", smiles=True)

gmso_top = mb_compound.to_gmso()

benzene_mol = Molecule(num_mols=20, compound=mb_compound)
benzene_mol = Molecule(num_mols=20, compound=gmso_top)


#### Option 4: Define a subclass of the `Molecule` class

Checkout some polymer examples defined in `flower/library/polymers.py`.

## 2. Forcefields

`flower` offers a range of predefined forcefield classes that can be used to parameterize a system. Checkout  [`flower/library/forcefields.py`](https://github.com/cmelab/flower/blob/main/flower/library/forcefields.py) to see the list of available forcefields.

To create a custom forcefield object, you can follow these two approaches:

### Custom ForceFields (XML-based)

Given the path to the XML file of a desired forcefield, users can employ the `FF_from_file` class available in `flower.library` to instantiate a forcefield object.

In [ ]:
from flower.library import FF_from_file

benzene_ff = FF_from_file(forcefield_files="../flower/assets/forcefields/benzene_opls.xml")

Check out [`flower/library/forcefields.py`](https://github.com/cmelab/flower/blob/main/flower/library/forcefields.py) for more some examples of defining a forcefield using a subclass of `foyer.Forcefield` for specific molecules.

### HOOMD Forces

Alternatively, users have the flexibility to define their own custom class that handles forces. This can be achieved by implementing a class or method that generates a list of `hoomd.md.force` objects, tailored to specific simulation requirements.
Check out the `BeadSpring` class in  [`flower/library/forcefields.py`](https://github.com/cmelab/flower/blob/main/flower/library/forcefields.py) for an example of defining HOOMD force objects for a system of coarse-grained beads.

## 3. System

### Custom system assembly algorithms

The `System` class provides two distinct methods for filling the simulation box: `Pack` and `Lattice`.

`Pack` fills the simulation box with molecules in a way that closely packs them, typically in a disordered, random fashion.

`Lattice`, on the other hand, fills the simulation box using a well-defined lattice or grid structure. The `x` and `y` parameters specify the distance (in nm) between lattice points in x and y directions respectively. The `n` parameter determines the number of times to repear the unit cell in x and y directions.

In [ ]:
# example of defining a system using the Lattice method
from flower.base import Lattice
from flower.library import PPS, OPLS_AA_PPS

pps = PPS(num_mols=32, lengths=5)

lattice = Lattice(molecules=pps, density=0.8, x=1, y=1, n=4)

In [ ]:
lattice.system.visualize()

Note: Both `Pack` and `Lattice` classes are subclasses of the `System` class. The base `System` class is designed as an abstract class, meaning it's not intended to be directly instantiated.

If you desire to customize a molecule assembly algorithm that suits your specific requirements, you should create a subclass of the `System` class and override the abstract method `_build_system`. This method is responsible for organizing molecules from the `Molecule` class into a simulation box and returning the resulting mbuild compound. 

Check out `Pack` and `Lattice` in `flower/base/systems.py` for examples of how to define custom assembly algorithms.

### Mixture of molecules

`flower` allows users to construct a system consisting of a mixture of molecule types, each potentially utilizing different force fields. If all the molecule types within the system share the same forcefield, then you only need to pass the forcefield once.

In [ ]:
from flower.base import Pack
from flower.library import OPLS_AA_DIMETHYLETHER
dimethylether_mol = Molecule(num_mols=20, smiles="COC")
pps_mol = PPS(num_mols=10, lengths=4)
multi_type_system = Pack( molecules=[dimethylether_mol, pps_mol], density=0.8)
multi_type_system.apply_forcefield(r_cut=2.5, force_field=[OPLS_AA_DIMETHYLETHER(), OPLS_AA_PPS()], auto_scale=True)

In [ ]:
multi_type_system.system.visualize()

### AutoScale & Reference Values

As mentioned in the previous tutorial, when the `auto_scale` parameter is set to `True` during system building, the parameters of the forcefields will be scaled automatically to range between 0 to 1. The scaling factor for these conversions can be retrieved from `system.reference_values`, which includes scaling factors for energy, length and mass values.

TODO: (need to talk about units and how to set units)

## 4. Simulation

### Simulation methods

Simulation methods/functionalities available in `flower` `Simulation` class:


- `adjust_epsilon`: scales or shifts all epsilon values of LJ forces by a specified value. 
- `adjust_sigma`: scales or shifts all sigma values of LJ forces by a specified value.
- `add_walls`, `remove_walls`: TODO
- `run_update_volume`: Shrinks or expands the box to a given length.
- `run_langevin`: runs simulation with Langevin dynamics.
- `run_NPT`: runs simulation with the NPT ensemble.
- `run_NVT`: runs simulation with the NVT ensemble.
- `run_NVE`: runs simulation with the NVE ensemble.
- `run_displacement_cap`: runs NVE simulation with a cap on the maximum distance travelled by particles.
- `pickle_forcefield`: create a pickle file to save the force objects. This can be used to recreate a simulation class without the need to initiate `Molecule` and `System` classes.
- `save_restart_gsd`: saves a snapshot of the current state of the system. This can be used to recreate a simulation without the need to initiate `Molecule` and `System` classes.

### Resume a simulation

Resuming a simulation in `flower` is easy, without the need to recreate the `Molecule` and `Simulation` classes. All that's required are two key components: a gsd snapshot of the system and a list of HOOMD force objects.

Users can save the snapshot of a simulation's latest state by invoking the `simulation.save_restart_gsd()` method. By default, this snapshot is saved in a file named `restart.gsd`. However, users have the flexibility to specify a different file path by configuring `file_path` parameter.

Additionaly, the list of HOOMD force objects can be preserved in a pickled format using `simulation.pickle_forcefield()` method. Pickled list is saved in `forcefield.pickle` by default. The file path can be configured with `file_path` parameter.
Please note that the user will need to pass the unpickled list to simulation class in order to recreate the class.

In [ ]:
from flower.base import Molecule, Pack, Simulation
from flower.library import OPLS_AA

molecule = Molecule(num_mols=30, smiles="c1ccccc1")

system = Pack(molecules=molecule, density=0.5)
system.apply_forcefield(r_cut=2.5, force_field=OPLS_AA(), auto_scale=True)

sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=system.hoomd_forcefield)

sim.run_NVE(n_steps=1e3)

In [ ]:
sim.save_restart_gsd("snapshot.gsd")

sim.pickle_forcefield("hoomd_forces.pickle")

In [ ]:
import pickle

f = open("hoomd_forces.pickle", "rb")
hoomd_ff = pickle.load(f)

In [ ]:
resume_sim = Simulation(initial_state="snapshot.gsd", forcefield=hoomd_ff)